In [1]:
import networkx as nx
G=nx.Graph()
G.add_edges_from([(1,2),(1,3),(1,4),(3,4)])
G

In [2]:
G.nodes(data=True)

NodeDataView({1: {}, 2: {}, 3: {}, 4: {}})

In [3]:
G.node[1]['attribute']='value'
G.nodes(data=True)

NodeDataView({1: {'attribute': 'value'}, 2: {}, 3: {}, 4: {}})

In [4]:
nx.write_graphml(G,'so.graphml')